In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftConfig, PeftModel

config  = PeftConfig.from_pretrained("/workspace/Emotion_Intent_Chat/emo_int_chat/intent_lora_tuning/tuned_model/intent_lora_Swallow-7b-instruct-v0.1_20240908_125838_acknowledging")
base_model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, device_map="auto", load_in_8bit=True).eval()
tokenizer = AutoTokenizer.from_pretrained("/workspace/Emotion_Intent_Chat/emo_int_chat/intent_lora_tuning/tuned_model/intent_lora_Swallow-7b-instruct-v0.1_20240908_125838_acknowledging")

device = "cuda"

model = PeftModel.from_pretrained(base_model, "/workspace/Emotion_Intent_Chat/emo_int_chat/intent_lora_tuning/tuned_model/intent_lora_Swallow-7b-instruct-v0.1_20240908_125838_acknowledging", adapter_name="acknowledging", is_trainable=False)
# _ = model.load_adapter("/workspace/Emotion_Intent_Chat/emo_int_chat/intent_lora_tuning/tuned_model/intent_lora_Swallow-7b-instruct-v0.1_20240909_102133_agreeing", adapter_name="agreeing", is_trainable=False)
# _ = model.load_adapter("/workspace/Emotion_Intent_Chat/emo_int_chat/intent_lora_tuning/tuned_model/intent_lora_Swallow-7b-instruct-v0.1_20240910_025103_consoling", adapter_name="consoling", is_trainable=False)
# _ = model.load_adapter("/workspace/Emotion_Intent_Chat/emo_int_chat/intent_lora_tuning/tuned_model/intent_lora_Swallow-7b-instruct-v0.1_20240910_230527_encouraging", adapter_name="encouraging", is_trainable=False)
# _ = model.load_adapter("/workspace/Emotion_Intent_Chat/emo_int_chat/intent_lora_tuning/tuned_model/intent_lora_Swallow-7b-instruct-v0.1_20240911_144251_questioning", adapter_name="questioning", is_trainable=False)
# _ = model.load_adapter("/workspace/Emotion_Intent_Chat/emo_int_chat/intent_lora_tuning/tuned_model/intent_lora_Swallow-7b-instruct-v0.1_20240912_190737_sympathizing", adapter_name="sympathizing", is_trainable=False)
# _ = model.load_adapter("/workspace/Emotion_Intent_Chat/emo_int_chat/intent_lora_tuning/tuned_model/intent_lora_Swallow-7b-instruct-v0.1_20240914_034959_wishing", adapter_name="wishing", is_trainable=False)
# _ = model.load_adapter("/workspace/Emotion_Intent_Chat/emo_int_chat/intent_lora_tuning/tuned_model/intent_lora_Swallow-7b-instruct-v0.1_20240921_200113_suggesting", adapter_name="suggesting", is_trainable=False)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
model.merge_adapter()
list(model.state_dict())


/opt/conda/envs/trl/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:83: UserWarning: Merge lora module to 8-bit linear may get different generations due to rounding errors.
  warnings.warn(


['base_model.model.model.embed_tokens.weight',
 'base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight',
 'base_model.model.model.layers.0.self_attn.q_proj.base_layer.SCB',
 'base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight_format',
 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.acknowledging.weight',
 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.acknowledging.weight',
 'base_model.model.model.layers.0.self_attn.k_proj.weight',
 'base_model.model.model.layers.0.self_attn.k_proj.SCB',
 'base_model.model.model.layers.0.self_attn.k_proj.weight_format',
 'base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight',
 'base_model.model.model.layers.0.self_attn.v_proj.base_layer.SCB',
 'base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight_format',
 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.acknowledging.weight',
 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.acknowledging.weight',
 'base_mode

In [9]:
import pickle
from tqdm import tqdm
with open(
        # "/workspace/Emotion_Intent_Chat/JEmpatheticDialogue/JEmpatheticDialogue.pkl",
        "/workspace/Emotion_Intent_Chat/JEmpatheticDialogue/JEmpatheticDialogue_1turn.pkl",
        "rb",
    ) as f:
        conversation_list = pickle.load(f)
conversation_prompt_list = []
for conversation in tqdm(conversation_list):
    conversation.insert(0, {"role": "system", "content": "ユーザの発話に対して共感し、寄り添うような返答を日本語でしてください。その際、一言から二言程度で短く端的に答えてください。"})
    conversation_prompt_list.append(conversation)

100%|██████████| 20000/20000 [00:00<00:00, 3639151.45it/s]


In [10]:

# model.add_weighted_adapter(
#     adapters=['acknowledging', 'agreeing', 'consoling', 'encouraging', 'questioning', 'sympathizing', 'wishing', 'suggesting'],
#     weights=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
#     adapter_name='merge',
#     density=1.0,
#     combination_type='ties'
# )

# model.set_adapter('merge')


for messages in conversation_prompt_list[:5]:
    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

    model_inputs = encodeds.to(device)
    # model.to(device)

    generated_ids = model.generate(model_inputs, max_new_tokens=512, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)
    print(decoded[0])


<s>[INST] <<SYS>>
ユーザの発話に対して共感し、寄り添うような返答を日本語でしてください。その際、一言から二言程度で短く端的に答えてください。
<</SYS>>

寝ているときに首がもぞもぞしたので、起きて見てみたらムカデがいたの！ [/INST] そっか、それはびっくりしたね！ムカデは危険な生き物だから、注意して扱ってね。</s>
<s>[INST] <<SYS>>
ユーザの発話に対して共感し、寄り添うような返答を日本語でしてください。その際、一言から二言程度で短く端的に答えてください。
<</SYS>>

車同士がぶつかる瞬間を見てしまいました。 [/INST] それは怖かったね。注意して運転してください。</s>
<s>[INST] <<SYS>>
ユーザの発話に対して共感し、寄り添うような返答を日本語でしてください。その際、一言から二言程度で短く端的に答えてください。
<</SYS>>

先日、うちの子が学校の帰りに体調悪くて座り込んでたおじいさんを助けて表彰されたんだ。 [/INST] それはとっても感心なことです！ 子どもの優しい心が、おじいさんの命を救ったんですね。 素晴らしい行動です！</s>
<s>[INST] <<SYS>>
ユーザの発話に対して共感し、寄り添うような返答を日本語でしてください。その際、一言から二言程度で短く端的に答えてください。
<</SYS>>

この間一人で焼肉屋さんに行ったんですけど、寂しくてあまり楽しくありませんでした。 [/INST] それはいいね、一人で焼肉を楽しめるのはいいことだよ！寂しくても、美味しいものを食べながら時間を過ごすのはいい気分転換になるよ。</s>
<s>[INST] <<SYS>>
ユーザの発話に対して共感し、寄り添うような返答を日本語でしてください。その際、一言から二言程度で短く端的に答えてください。
<</SYS>>

先週、車を買ったんです。 [/INST] それはいいね！車を持っているのは、移動が便利になるし、運転も楽しいよ！</s>


100%|██████████| 20000/20000 [00:00<00:00, 3541439.61it/s]


In [4]:
conversation_prompt_list  

[[{'role': 'system',
   'content': 'ユーザの発話に対して共感し、寄り添うような返答を日本語でしてください。その際、一言から二言程度で短く端的に答えてください。'},
  {'role': 'user', 'content': '寝ているときに首がもぞもぞしたので、起きて見てみたらムカデがいたの！'}],
 [{'role': 'system',
   'content': 'ユーザの発話に対して共感し、寄り添うような返答を日本語でしてください。その際、一言から二言程度で短く端的に答えてください。'},
  {'role': 'user', 'content': '車同士がぶつかる瞬間を見てしまいました。'}],
 [{'role': 'system',
   'content': 'ユーザの発話に対して共感し、寄り添うような返答を日本語でしてください。その際、一言から二言程度で短く端的に答えてください。'},
  {'role': 'user', 'content': '先日、うちの子が学校の帰りに体調悪くて座り込んでたおじいさんを助けて表彰されたんだ。'}],
 [{'role': 'system',
   'content': 'ユーザの発話に対して共感し、寄り添うような返答を日本語でしてください。その際、一言から二言程度で短く端的に答えてください。'},
  {'role': 'user', 'content': 'この間一人で焼肉屋さんに行ったんですけど、寂しくてあまり楽しくありませんでした。'}],
 [{'role': 'system',
   'content': 'ユーザの発話に対して共感し、寄り添うような返答を日本語でしてください。その際、一言から二言程度で短く端的に答えてください。'},
  {'role': 'user', 'content': '先週、車を買ったんです。'}],
 [{'role': 'system',
   'content': 'ユーザの発話に対して共感し、寄り添うような返答を日本語でしてください。その際、一言から二言程度で短く端的に答えてください。'},
  {'role': 'user', 'content': '最近胸のあたりが痛い事が多いんだよ。'}],
 [{'role': '